In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
"""
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
"""
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n"

In [2]:
import tensorflow as tf
import seaborn as sns
from tensorflow.keras import layers
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("float32")

In [3]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
train_dir='../data/original/archive/train'
test_dir='../data/original/archive/test'
train_data=image_dataset_from_directory(train_dir,batch_size=32,image_size=(224,224),label_mode='categorical',shuffle=True,seed=42)
test_data=image_dataset_from_directory(test_dir,batch_size=32,image_size=(224,224),label_mode='categorical',shuffle=False,seed=42)


Found 15557 files belonging to 23 classes.
Found 4002 files belonging to 23 classes.


In [4]:
class_names=train_data.class_names
class_count=len(class_names)
print(len(class_names))
print(class_names)
train_data = train_data.prefetch(buffer_size = tf.data.AUTOTUNE)
test_data = test_data.prefetch(buffer_size = tf.data.AUTOTUNE)


23
['Acne_and_Rosacea', 'Actinic_Keratosis_Basal_Cell_Carcinoma_and_other_Malignant_Lesions', 'Atopic_Dermatitis', 'Bullous_Disease', 'Cellulitis_Impetigo_and_other_Bacterial_Infections', 'Eczema', 'Exanthems_and_Drug_Eruptions', 'Hair_Loss_Alopecia_and_other_Hair_Diseases', 'Herpes_HPV_and_other_STDs', 'Light_Diseases_and_Disorders_of_Pigmentation', 'Lupus_and_other_Connective_Tissue_diseases', 'Melanoma_Skin_Cancer_Nevi_and_Moles', 'Nail_Fungus_and_other_Nail_Disease', 'Poison_Ivy_and_other_Contact_Dermatitis', 'Psoriasis_Lichen_Planus_and_related_diseases', 'Scabies_Lyme_Disease_and_other_Infestations_and_Bites', 'Seborrheic_Keratoses_and_other_Benign_Tumors', 'Systemic_Disease', 'Tinea_Ringworm_Candidiasis_and_other_Fungal_Infections', 'Urticaria_Hives', 'Vascular_Tumors', 'Vasculitis', 'Warts_Molluscum_and_other_Viral_Infections']


In [5]:
from tensorflow.keras.layers import Dense,Activation,Dropout,Conv2D,MaxPooling2D,BatchNormalization,Flatten
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, load_model, Sequential
model_name='EfficientNetB2'
base_model=tf.keras.applications.EfficientNetB2(include_top=False, weights="imagenet",input_shape=(224,224,3), pooling='max') 
x=base_model.output
#x=layers.GlobalAvgPool2D(name = "pooling_layer")(x)
x=tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
x=Dense(256, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006),
                bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)
x=Dense(128, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006),
                bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)
x=Dropout(rate=.45, seed=42)(x)        
output=Dense(class_count, activation='softmax')(x)
model=Model(inputs=base_model.input, outputs=output)
model.compile(tf.keras.optimizers.Adamax(lr=.001), loss='categorical_crossentropy', metrics=['accuracy']) 

/Users/joshvirnarula/miniconda3/envs/tensorflow_ml/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adamax.py:95: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adamax, self).__init__(name, **kwargs)


In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 tf.math.truediv (TFOpLambda)   (None, 224, 224, 3)  0           ['normalization[0][0]']      

In [7]:
history=model.fit(train_data,epochs = 10,
                     validation_data = test_data)

Epoch 1/10


2023-04-08 14:18:09.613599: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


  8/487 [..............................] - ETA: 20:12 - loss: 14.2658 - accuracy: 0.0664

KeyboardInterrupt: 

In [ ]:
history.history

In [ ]:
import matplotlib.pyplot as plt
epochs=[]
for i in range(1,11):
    epochs.append(i)
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.figure(figsize=(8,3))

plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.ylim(0, 1)
plt.title('Training and validation cat accuracy')
plt.legend()



In [ ]:

pred_probs = model.predict(test_data)
pred_classes = pred_probs.argmax(axis =1)
y_labels = []
for image,label in test_data.unbatch():
    y_labels.append(label.numpy().argmax())
    

In [ ]:
!pip install --force-reinstall sklearn
import sklearn
from sklearn.metrics import classification_report
print("Classification report\n",classification_report(y_labels,pred_classes))

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
import matplotlib.image as mping
def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(10, 10), text_size=15, norm=False, savefig=False): 
    cm = confusion_matrix(y_true, y_pred)
    cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # normalize it
    n_classes = cm.shape[0] # find the number of classes we're dealing with

  # Plot the figure and make it pretty
    fig, ax = plt.subplots(figsize=figsize)
    cax = ax.matshow(cm, cmap=plt.cm.Blues) # colors will represent how 'correct' a class is, darker == better
    fig.colorbar(cax)

  # Are there a list of classes?
    if classes:
        labels = classes
    else:
        labels = np.arange(cm.shape[0])
    ax.set(title="Confusion Matrix",
         xlabel="Predicted label",
         ylabel="True label",
         xticks=np.arange(n_classes), # create enough axis slots for each class
         yticks=np.arange(n_classes), 
         xticklabels=labels, # axes will labeled with class names (if they exist) or ints
         yticklabels=labels)
  
  # Make x-axis labels appear on bottom
    ax.xaxis.set_label_position("bottom")
    ax.xaxis.tick_bottom()

  ### Added: Rotate xticks for readability & increase font size (required due to such a large confusion matrix)
    plt.xticks(rotation=70, fontsize=text_size)
    plt.yticks(fontsize=text_size)
    threshold = (cm.max() + cm.min()) / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if norm:
            plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
            horizontalalignment="center",
            color="white" if cm[i, j] > threshold else "black",
            size=text_size)
        else:
            plt.text(j, i, f"{cm[i, j]}",
            horizontalalignment="center",
            color="white" if cm[i, j] > threshold else "black",
            size=text_size)

  # Save the figure to the current working directory
    if savefig:
        fig.savefig("confusion_matrix.png")

In [ ]:
make_confusion_matrix(y_labels,pred_classes,classes = class_names,figsize = (20,20))

In [ ]:
subject='Skin'
save_path= r'./'
acc=87
save_id=str (model_name +  '-' + subject +'-'+ str(acc)[:str(acc).rfind('.')+3] + '.h5')
model_save_loc=os.path.join(save_path, save_id)
model.save(model_save_loc)

In [ ]:
import pickle
pickle.dump(model, open('model.pkl', 'wb'))

In [ ]:
"""
import joblib
from joblib import Parallel, delayed
joblib.dump(model, 'EFFNETB2_87.pkl')
"""

In [ ]:
acc=history.history["accuracy"]
loss=history.history["loss"]
val_acc=history.history["val_accuracy"]
val_loss=history.history["val_loss"]
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')

In [ ]:
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')